<h2>This script is for internal developer testing.</h2>

In [1]:
import wfdb

help(wfdb.Record.__init__)

Help on function __init__ in module wfdb.records:

__init__(self, p_signals=None, d_signals=None, recordname=None, nsig=None, fs=None, counterfreq=None, basecounter=None, siglen=None, basetime=None, basedate=None, filename=None, fmt=None, sampsperframe=None, skew=None, byteoffset=None, adcgain=None, baseline=None, units=None, adcres=None, adczero=None, initvalue=None, checksum=None, blocksize=None, signame=None, comments=None)
    Initialize self.  See help(type(self)) for accurate signature.



In [1]:
import wfdb
# Read part of a record from Physiobank
sig, fields = wfdb.srdsamp('a103l', sampfrom = 50000, channels = [0,1], pbdir = 'challenge/2015/training')
# Write a local WFDB record (manually inserting fields)
wfdb.wrsamp('ecgrecord', fs = 250, units = ['mV', 'mV'], signames = ['I', 'II'], p_signals = sig, fmt = ['16', '16'])

checking field:  p_signals
inner ch:  None
checking field:  fmt
inner ch:  None


TypeError: list indices must be integers or slices, not NoneType

In [3]:
sig

array([[-0.0709259 ,  0.8128327 ],
       [-0.06733821,  0.81872624],
       [-0.06292259,  0.82851711],
       ..., 
       [-0.04084449,  0.7493346 ],
       [-0.04719194,  0.7581749 ],
       [-0.04677798,  0.7615019 ]])

In [4]:

fields

{'comments': ['Asystole', 'False alarm'],
 'fs': 250.0,
 'signame': ['II', 'V'],
 'units': ['mV', 'mV']}

In [2]:
list(range(len([1,2,3])))

[0, 1, 2]